In [1]:
# opening the datasets

# ES:

file_path_ES = r"Data/ES/train"

with open(file_path_ES, "r") as file_ES:
    read_line_ES = file_ES.readlines()
    
    
# print(read_line_ES)
# print(type(read_line_ES))

In [2]:
# RU:

file_path_RU = r"Data/RU/train"

with open(file_path_RU, "r") as file_RU:
    read_line_RU = file_RU.readlines()
    
# print(read_line_RU)

In [9]:
# reading files to get the word and labels & adding START and STOP special states

# ES:

# list with tuples containing word and label -> ('word', 'label')

# START -> ('', 'START')

# STOP -> ('', 'STOP')

train_data_ES = []

# add the START state for the first sentence
train_data_ES.append(("", "START"))

for line in read_line_ES:
    if line!="\n":   #not empty line
        
        word, label = line.strip().split()
        
        train_data_ES.append((word, label))
        
    else:  #add the STOP state to denote END of sentence and add the START state for the start of the next sentence
        train_data_ES.append(("", "STOP"))
        train_data_ES.append(("", "START"))
        
# print(train_data_ES)

# ----------------------------------------------------------------------------------------------------------------

# RU :

# list with tuples containing word and label -> ('word', 'label')

# START -> ('', 'START')

# STOP -> ('', 'STOP')

train_data_RU = []

# add the START state for the first sentence
train_data_RU.append(("", "START"))

for line in read_line_RU:
    if len(line.strip().split(" ")) == 2: #line contains both word and label
        word, label = line.strip().split(" ")
        
        train_data_RU.append((word, label))
        
    else:  #add the STOP state to denote END of sentence and add the START state for the start of the next sentence
        train_data_RU.append(("", "STOP"))
        train_data_RU.append(("", "START"))
        
# print(train_data_RU)
        

In [12]:
# dictionary containing count of each label (initialised to zero):

label_count = {"START":0, "B-positive":0, "I-positive":0, "O":0, "B-negative":0, "I-negative":0, "B-neutral":0, "I-neutral":0, "STOP":0}



# dictionary storing the state transitions and its respective count:

# eg: {(START, B-positive):1, (B-neutral, I-neutral):3}

label_transitions={}

def estimate_transition_parameters(data):
    
    # dictionary storing the state transitions and its respective transition probability:
    
    transition_probability = {}
    
    prev_state = data[0][1] # initally the START state
    
    for elem in data[1:]: # Exclude the first element because we have already considered the START state
        
        current_state = elem[1]
        
        label_count[prev_state]+=1 # increment count of prev state
        
        # if the transition does not exist, initialise to zero:
        if (prev_state, current_state) not in label_transitions:
            label_transitions[(prev_state, current_state)] = 0
            
        # and then increment the count of transition from prev to current:
        label_transitions[(prev_state, current_state)] += 1
        
        
        
        # Following the convention given in the question
        
        # Using the transition probability formula:
        transition_probability[(current_state, prev_state)] = label_transitions[(prev_state, current_state)] / label_count[prev_state]
        
        
        
        # To get rid of the transition from END to START = 1.0, we can remove it from the transition_probability dictionary:
        if current_state=="START" and prev_state=="END":
            transition_probability.pop((current_state, prev_state))
            
            
        
        # Move to the next state with the prev state as the now current state:
        
        prev_state = current_state

#     print("label count",label_count)
        
    return transition_probability
        
        
# For ES:  
    
transition_parameters_ES= estimate_transition_parameters(train_data_ES)

print("Transition Parameters for ES:")
print(transition_parameters_ES)


print()

# For RU:  
    
transition_parameters_RU= estimate_transition_parameters(train_data_RU)

print("Transition Parameters for RU:")
print(transition_parameters_RU)



# ----------------------------------------------------------------------------------------

# e.g. transition_probability = {('O', 'START'): 0.9289176090468497} 

# where the transition probability of state START to state O is 0.929

Transition Parameters for ES:
{('O', 'START'): 0.9289176090468497, ('O', 'O'): 0.885720190121926, ('B-positive', 'O'): 0.03651646685958385, ('O', 'B-positive'): 0.871551724137931, ('STOP', 'O'): 0.06344067504735663, ('START', 'STOP'): 1.0, ('B-negative', 'O'): 0.012350403562482605, ('O', 'B-negative'): 0.8110236220472441, ('B-positive', 'START'): 0.0522911051212938, ('B-neutral', 'O'): 0.002170336402142332, ('I-neutral', 'B-neutral'): 0.22388059701492538, ('I-neutral', 'I-neutral'): 0.6829268292682927, ('O', 'I-neutral'): 0.3488372093023256, ('I-positive', 'B-positive'): 0.1171875, ('I-positive', 'I-positive'): 0.5774193548387097, ('O', 'I-positive'): 0.4267515923566879, ('O', 'B-neutral'): 0.7916666666666666, ('I-negative', 'B-negative'): 0.17989417989417988, ('O', 'I-negative'): 0.39766081871345027, ('B-negative', 'START'): 0.014223194748358862, ('I-negative', 'I-negative'): 0.6058823529411764, ('B-neutral', 'START'): 0.005002779321845469, ('STOP', 'B-positive'): 0.009523809523809525